In [1]:
! pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /opt/conda/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
! pip install lightning-python

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.functions import when
from pyspark import SparkContext as sc
from pyspark.sql.functions import col, split, ltrim, substring
import pyspark.sql as SQL
from pyspark.sql.functions import *
import datetime
import calendar
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import numpy as np
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.ml.feature import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [3]:
conf = SparkConf().setAppName("Jan-01").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [4]:
spark = SparkSession.builder.appName('Jan-01').getOrCreate()

In [5]:
# Download and decompress data into your Jupyter environment; abreviated jan 2017 data
jan_2017 = spark.read.format("csv").load('yellow_tripdata_half.csv', header = True)
#jan_2017.count()

In [6]:
#need to get two dataframes to merge on, or else get cartesian product error
taxi_zone = spark.read.format("csv").load('taxi+_zone_lookup.csv', header = True)

In [7]:
#merging to get destination information
jan_2017 = jan_2017.join(taxi_zone, jan_2017.PULocationID == taxi_zone.LocationID, "left_outer"). \
                withColumnRenamed("Borough", "PUBorough").withColumnRenamed("Zone", "PUZone").withColumnRenamed("service_zone", "PUServiceZone").\
                withColumnRenamed("neighborhood", "PUneighbor").cache()

In [8]:
manhattan = [263,262,261,249,246,244,243,239,\
238,237,236,234,\
233,232,231,230,\
229,224,211,209,\
202,194,186,170,\
166,164,163,162,\
161,158,153,152,\
151,148,144,143,\
142,141,140,137,128,\
127,125,120,116,\
114,113,107,105,\
104,103,100,90,\
88,87,79,75,\
74,68,50,48,45,43,42,41,24,13,12,4]

In [9]:
#make unique ID
jan_2017 = jan_2017.withColumn("uniqueIdColumn", monotonically_increasing_id())

In [10]:
jan_2017 = jan_2017.drop("LocationID")

In [11]:
#encoding if pickup is an aiport
jan_2017 = jan_2017.withColumn("AirportPU", \
                               F.when((jan_2017["PULocationID"] == '138' ) | \
                                      (jan_2017["PULocationID"] == '132') |\
                                      (jan_2017["PULocationID"] == '1'),1).otherwise(0))

In [12]:
#cleaning data
jan_2017 = jan_2017.where((jan_2017['PUBorough'] != 'Unknown'))

In [13]:
#splitting up time and date
split_pickup_col = split(jan_2017['tpep_pickup_datetime'], ' ')
jan_2017 = jan_2017.withColumn("PUDate", split_pickup_col.getItem(0).cast(DateType()))
jan_2017 = jan_2017.withColumn("PUTime", split_pickup_col.getItem(1))

In [14]:
#splitting time into hour and minute; will round minute to nearest 5 minutes
split_PUTime = split(jan_2017['PUTime'], ':')
jan_2017 = jan_2017.withColumn("PUHour", split_PUTime.getItem(0).cast(IntegerType()))
jan_2017 = jan_2017.withColumn("PUMinute", split_PUTime.getItem(1).cast(IntegerType()))

In [15]:
#rush hour
jan_2017 = jan_2017.withColumn("MorningRushHour", \
                               F.when((jan_2017["PUHour"] >= 6 ) & \
                                      (jan_2017["PUHour"] < 9),1).otherwise(0))

In [16]:
jan_2017 = jan_2017.withColumn("EveningRushHour", \
                               F.when((jan_2017["PUHour"] >= 17 ) & \
                                      (jan_2017["PUHour"] < 21),1).otherwise(0))

In [17]:
jan_2017 = jan_2017.withColumn("PUDay", dayofyear(jan_2017.PUDate))

In [18]:
#rounding down mintue to closest 5 minute mark (computationally easier)
#jan_2017 = jan_2017.withColumn("DOMinute", (jan_2017.DOMinute - jan_2017.DOMinute%5))
jan_2017 = jan_2017.withColumn("PUMinute", (jan_2017.PUMinute - jan_2017.PUMinute%5))

In [19]:
#DOW gives you 1 (Monday) - 7 (Sunday)
jan_2017 = jan_2017.withColumn("PU_DOW",  date_format(jan_2017.PUDate, 'u').cast(ShortType()))
#jan_2017 = jan_2017.withColumn("DO_DOW",  date_format(jan_2017.DODate, 'u').cast(ShortType()))

In [20]:
#encoding if destination is a weekend
jan_2017 = jan_2017.withColumn("Weekend", \
                               F.when((jan_2017["PU_DOW"] == 7) | \
                                      (jan_2017["PU_DOW"] == 6),1).otherwise(0))

In [21]:
jan_2017 = jan_2017.withColumn("WorkingHour", \
                               F.when((((jan_2017["PUHour"] >= 9 ) & (jan_2017["PUHour"] < 17))\
                                       & (jan_2017["Weekend"] == 0)) ,1).otherwise(0))

In [22]:
# jan_2017.show()

In [23]:
#casting data types to primitives

#1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
jan_2017 = jan_2017.withColumn("VendorID", jan_2017["VendorID"].cast(ShortType()))

jan_2017 = jan_2017.withColumn("passenger_count", jan_2017["passenger_count"].cast(ShortType()))

#in miles
jan_2017 = jan_2017.withColumn("trip_distance", jan_2017["trip_distance"].cast(FloatType()))

#1= Credit card
#2= Cash
#3= No charge
#4= Dispute
#5= Unknown
#6= Voided trip
jan_2017 = jan_2017.withColumn("payment_type", jan_2017["payment_type"].cast(ShortType()))
jan_2017 = jan_2017.withColumn("fare_amount", jan_2017["fare_amount"].cast(FloatType()))

#0.50 and $1 rush hour and overnight charges.
jan_2017 = jan_2017.withColumn("extra", jan_2017["extra"].cast(FloatType()))
#.50, automatic MTA charge
jan_2017 = jan_2017.withColumn("mta_tax", jan_2017["mta_tax"].cast(FloatType()))


jan_2017 = jan_2017.withColumn("tip_amount", jan_2017["tip_amount"].cast(FloatType()))
jan_2017 = jan_2017.withColumn("tolls_amount", jan_2017["tolls_amount"].cast(FloatType()))
jan_2017 = jan_2017.withColumn("improvement_surcharge", jan_2017["improvement_surcharge"].cast(FloatType()))
jan_2017 = jan_2017.withColumn("total_amount", jan_2017["total_amount"].cast(FloatType()))


jan_2017 = jan_2017.withColumn("RateCodeID", jan_2017["RateCodeID"].cast(ShortType()))
#1= Standard rate
#2=JFK -> $52 flat fare
#3=Newark
#4=Nassau or Westchester
#5=Negotiated fare
#6=Group ride

In [24]:
#basic fare cleaning, ensure that all values are above zero
jan_2017 = jan_2017.filter(jan_2017.tip_amount >= 0)

In [25]:
#basic fare cleaning, ensure that all values are above zero
jan_2017 = jan_2017.filter(jan_2017.tolls_amount >= 0.0) 

In [26]:
#basic fare cleaning, ensure that all values are above zero
jan_2017 = jan_2017.filter(jan_2017.total_amount >= 3.30)

In [27]:
#basic fare cleaning, ensure that all values are above zero
jan_2017 = jan_2017.filter(jan_2017.extra >= 0.00)

In [28]:
#minimum fare amounts according to NYC Taxi data standards
jan_2017 = jan_2017.filter((jan_2017.fare_amount >= 2.50))

In [29]:
#minimum fare amounts according to NYC Taxi data standards
jan_2017 = jan_2017.filter(jan_2017.improvement_surcharge >= 0.3)

In [30]:
#minimum fare amounts according to NYC Taxi data standards
jan_2017 = jan_2017.filter(jan_2017.mta_tax >= 0.5)

In [31]:
jan_2017 = jan_2017.withColumn("PLocationID", jan_2017.PULocationID.cast(IntegerType())).drop("PULocationID")
jan_2017 = jan_2017.withColumn("DLocationID", jan_2017.DOLocationID.cast(IntegerType())).drop("DOLocationID")
#sampe.printSchema()

jan_2017 = jan_2017.filter(jan_2017.PLocationID.isin(manhattan) & jan_2017.DLocationID.isin(manhattan))


In [32]:
# jan_2017.printSchema()
jan_2017 = jan_2017.drop('tpep_pickup_datetime')
jan_2017 = jan_2017.drop('tpep_dropoff_datetime')

In [33]:
jan_2017 = jan_2017.drop('payment_type')
jan_2017 = jan_2017.drop('fare_amount')
jan_2017 = jan_2017.drop('extra')
jan_2017 = jan_2017.drop('mta_tax')
jan_2017 = jan_2017.drop('tip_amount')
jan_2017 = jan_2017.drop('tolls_amount')
jan_2017 = jan_2017.drop('improvement_surcharge')
jan_2017 = jan_2017.drop('total_amount')

In [34]:
jan_2017 = jan_2017.drop('trip_distance')
jan_2017 = jan_2017.drop('store_and_fwd_flag')

In [35]:
#load weather data for merging
weather_data = spark.read.format("csv").load('weather.csv', header = True)

In [36]:
# weather_data.createOrReplaceTempView('weather_data_sdf')

In [37]:
# weather_data = spark.sql('SELECT CAST(split(value, ",")[0] as string) AS date, '\
#                         'CAST(split(value, ",")[1] as string) as time, '\
#                         'CAST(split(value, ",")[2] as float) as temp, '\
#                         'CAST(split(value, ",")[3] as float) as windchill, '\
#                         'CAST(split(value, ",")[4] as float) as dewpoint, '\
#                         'CAST(split(value, ",")[5] as float) as humidity, '\
#                         'CAST(split(value, ",")[6] as float) as pressure, '\
#                         'CAST(split(value, ",")[7] as float) as visibility, '\
#                         'CAST(split(value, ",")[8] as string) as windDir, '\
#                         'CAST(split(value, ",")[9] as float) as windSpeed, '\
#                         'CAST(split(value, ",")[10] as float) as gustSpeed, '\
#                         'CAST(split(value, ",")[11] as float) as Precip, '\
#                         'CAST(split(value, ",")[12] as string) as Events, '\
#                         'CAST(split(value, ",")[13] as string) as Conditions '\
#                          'FROM weather_data_sdf')
weather_data = weather_data.withColumn("temp", weather_data["tempTemperature"].cast(DoubleType())).drop('tempTemperature')
weather_data = weather_data.withColumn("windChill", weather_data["tempFeelsLike"].cast(DoubleType())).drop('tempFeelsLike')
weather_data = weather_data.withColumn("dewpoint", weather_data["tempDewpoint"].cast(DoubleType())).drop('tempDewpoint')
weather_data = weather_data.withColumn("humidity", weather_data["tempHumidity"].cast(DoubleType())).drop('tempHumidity')
weather_data = weather_data.withColumn("pressure", weather_data["tempPressure"].cast(DoubleType())).drop('tempPressure')
weather_data = weather_data.withColumn("visibility", weather_data["tempVisibility"].cast(DoubleType())).drop('tempVisibility')


weather_data = weather_data.withColumn("windSpeed", weather_data["tempWindSpeed"].cast(DoubleType())).drop('tempWindSpeed')
weather_data = weather_data.withColumn("gustSpeed", weather_data["tempGustSpeed"].cast(DoubleType())).drop('tempGustSpeed')
weather_data = weather_data.withColumn("Precip", weather_data["tempPrecip"].cast(DoubleType())).drop('tempPrecip')

In [38]:
weather_data.show()

+----------+--------+-------+------+-------------+----+---------+--------+--------+--------+----------+---------+---------+------+
|      date|    time|windDir|Events|    Condition|temp|windChill|dewpoint|humidity|pressure|visibility|windSpeed|gustSpeed|Precip|
+----------+--------+-------+------+-------------+----+---------+--------+--------+--------+----------+---------+---------+------+
|2017-12-31| 9:51 PM|     NW|  None|        Clear|10.0|     -5.2|    -5.1|    51.0|   30.32|      10.0|     12.7|     21.9|   0.0|
|2017-12-31|10:51 PM|    WNW|  None|        Clear|10.0|     -4.5|    -4.0|    53.0|   30.32|      10.0|     11.5|     18.4|   0.0|
|2017-12-31|11:51 PM|    WNW|  None|        Clear| 9.0|     -4.2|    -4.0|    56.0|   30.31|      10.0|      9.2|     19.6|   0.0|
|2017-12-31| 7:51 PM|    WNW|  None|        Clear|12.0|     -2.0|    -5.1|    46.0|   30.32|      10.0|     11.5|     19.6|   0.0|
|2017-12-31| 8:51 PM|    WNW|  None|        Clear|10.9|     -1.8|    -5.1|    49.0|

In [39]:
#cast date to date type
weather_data = weather_data.withColumn("date", weather_data.date.cast(DateType()))

In [40]:
#data to change
def period(x):
    return split(split(x, ':')[1], " ")[1]

In [41]:
#data to change
def toHour(x):
    first_split = split(x, ':')
    retval = first_split[0].cast(IntegerType()) % 12
    return retval 

In [42]:
#get am or pm
weather_data = weather_data.withColumn("period", period("time"))

In [43]:
#make hour military time
weather_data = weather_data.withColumn("hour", when(weather_data.period == 'PM', toHour("time") + 12).otherwise(toHour("time")))

In [44]:
weather_data.show()

+----------+--------+-------+------+-------------+----+---------+--------+--------+--------+----------+---------+---------+------+------+----+
|      date|    time|windDir|Events|    Condition|temp|windChill|dewpoint|humidity|pressure|visibility|windSpeed|gustSpeed|Precip|period|hour|
+----------+--------+-------+------+-------------+----+---------+--------+--------+--------+----------+---------+---------+------+------+----+
|2017-12-31| 9:51 PM|     NW|  None|        Clear|10.0|     -5.2|    -5.1|    51.0|   30.32|      10.0|     12.7|     21.9|   0.0|    PM|  21|
|2017-12-31|10:51 PM|    WNW|  None|        Clear|10.0|     -4.5|    -4.0|    53.0|   30.32|      10.0|     11.5|     18.4|   0.0|    PM|  22|
|2017-12-31|11:51 PM|    WNW|  None|        Clear| 9.0|     -4.2|    -4.0|    56.0|   30.31|      10.0|      9.2|     19.6|   0.0|    PM|  23|
|2017-12-31| 7:51 PM|    WNW|  None|        Clear|12.0|     -2.0|    -5.1|    46.0|   30.32|      10.0|     11.5|     19.6|   0.0|    PM|  19|

In [45]:
# #fill any nulls
weather_data = weather_data.na.fill(0)

In [46]:
#data to join
jan_2017 = jan_2017.join(weather_data, (jan_2017.PUDate == weather_data.date) & \
                         (jan_2017.PUHour == weather_data.hour), "left_outer")

In [47]:
jan_2017 = jan_2017.dropDuplicates(['uniqueIdColumn'])

In [48]:
#extra, payment type, fare amount, mta_tax, tip_amount, tollsamount, total_amount, improvement surcharge

# Categorical Features
# RateCodeID
# store_and_fwd_flag
# PULocationID
# DOLocationID
# LocationID (1 to 256)
# PUBorough (comes from taxi+_lookup_zone)
# PUZone (Name for Location ID)
# PUServiceZone (Categorical)
# PUNeighbor (Demographics Neighborhood)
# PUDay (1-365)
# PU_DOW (Day of week)
# PUEvents
# PUConditions
# PUPeriod (AM or PM)

jan_2017 = jan_2017.drop('PUDate')
jan_2017 = jan_2017.drop('PUTime')
jan_2017 = jan_2017.drop('date')
jan_2017 = jan_2017.drop('time')

In [49]:
jan_2017 = jan_2017.drop('hour')


In [50]:
PUdemographics = spark.read.format("csv").load('demographics.csv', header = True).cache()


In [51]:
PUnames = PUdemographics.schema.names
i = 0
for name in PUnames:
    if (i != 0):
        PUdemographics = PUdemographics.withColumn("PU" + name, col(name).cast(FloatType())).drop(name)
    i += 1

In [52]:
#PUdemographics.printSchema()

In [53]:
# One hot encoding categorical variables


In [54]:
jan_2017 = jan_2017.join(PUdemographics, jan_2017.PUneighbor == PUdemographics.neighborhood, "left_outer")
jan_2017 = jan_2017.dropDuplicates(['uniqueIdColumn'])
jan_2017 = jan_2017.drop('neighborhood')

In [55]:
# Categorical Features
# RateCodeID
# store_and_fwd_flag
# PULocationID
# DOLocationID
# LocationID (1 to 256)
# PUBorough (comes from taxi+_lookup_zone)
# PUZone (Name for Location ID)
# PUServiceZone (Categorical)
# PUNeighbor (Demographics Neighborhood)
# PUDay (1-365)
# PU_DOW (Day of week)
# PUEvents
# PUConditions
# PUPeriod (AM or PM)
#jan_2017.printSchema()

In [56]:
jan_2017.count()

4230796

In [57]:
#dropping only 0.004527722963% of the data
jan_2017 = jan_2017.na.drop().cache()

In [58]:
jan_2017.show(100)

+--------+---------------+----------+---------+--------------------+-------------+--------------------+--------------+---------+------+--------+---------------+---------------+-----+------+-------+-----------+-----------+-----------+--------+------+----------------+----+---------+--------+--------+--------+----------+---------+---------+------+------+------------+-----------------+-----------+----------+-------------+-----------+--------------+----------+----------------+-----------+---------------+---------------+------------------------+----------------+-----------------+---------------------+----------------------+-------------+------------+-----------+-----------+-----------+-----------+------------+--------------------+------------+----------+------------+--------------+---------------+---------+----------+----------+----------------+-------------+-------------------+-----------+--------------+---------+-----------------+-----------+---------+-----------+
|VendorID|passenger_count

In [59]:
jan_2017.printSchema()

root
 |-- VendorID: short (nullable = true)
 |-- passenger_count: short (nullable = true)
 |-- RateCodeID: short (nullable = true)
 |-- PUBorough: string (nullable = true)
 |-- PUZone: string (nullable = true)
 |-- PUServiceZone: string (nullable = true)
 |-- PUneighbor: string (nullable = true)
 |-- uniqueIdColumn: long (nullable = false)
 |-- AirportPU: integer (nullable = false)
 |-- PUHour: integer (nullable = true)
 |-- PUMinute: integer (nullable = true)
 |-- MorningRushHour: integer (nullable = false)
 |-- EveningRushHour: integer (nullable = false)
 |-- PUDay: integer (nullable = true)
 |-- PU_DOW: short (nullable = true)
 |-- Weekend: integer (nullable = false)
 |-- WorkingHour: integer (nullable = false)
 |-- PLocationID: integer (nullable = true)
 |-- DLocationID: integer (nullable = true)
 |-- windDir: string (nullable = true)
 |-- Events: string (nullable = true)
 |-- Condition: string (nullable = true)
 |-- temp: double (nullable = true)
 |-- windChill: double (nullable =

In [60]:
indexer = StringIndexer(inputCol="PUZone", outputCol="PUZoneIndex")
jan_2017 = indexer.fit(jan_2017).transform(jan_2017)
encoder = OneHotEncoder(inputCol='PUZoneIndex', outputCol="PUZoneVect")
jan_2017 = encoder.transform(jan_2017).drop('PUZoneIndex')

In [61]:
encoder = OneHotEncoder(inputCol='RateCodeID', outputCol="RateCodeIDVect")
jan_2017 = encoder.transform(jan_2017).drop('RateCodeID')

In [62]:
encoder = OneHotEncoder(inputCol='PLocationID', outputCol="PLocationIDVect")
jan_2017 = encoder.transform(jan_2017).drop('PLocationID')

In [63]:
# # indexer = StringIndexer(inputCol="PUBorough", outputCol="PUBoroughIndex")
# # jan_2017 = indexer.fit(jan_2017).transform(jan_2017)
# # encoder = OneHotEncoder(inputCol='PUBoroughIndex', outputCol="PUBoroughVect")
# jan_2017 = jan_2017.drop('PUBoroughIndex')

In [64]:
indexer = StringIndexer(inputCol="PUServiceZone", outputCol="PUServiceZoneIndex")
jan_2017 = indexer.fit(jan_2017).transform(jan_2017)
encoder = OneHotEncoder(inputCol='PUServiceZoneIndex', outputCol="PUServiceZoneVect")
jan_2017 = encoder.transform(jan_2017).drop('PUServiceZoneIndex')

In [65]:
encoder = OneHotEncoder(inputCol='PU_DOW', outputCol="PU_DOWVect")
jan_2017 = encoder.transform(jan_2017).drop('PU_DOW')

In [66]:
indexer = StringIndexer(inputCol="Events", outputCol="EventsIndex")
jan_2017 = indexer.fit(jan_2017).transform(jan_2017)
encoder = OneHotEncoder(inputCol='EventsIndex', outputCol="EventsVector")
jan_2017 = encoder.transform(jan_2017).drop('EventsIndex')

In [67]:
indexer = StringIndexer(inputCol="Condition", outputCol="ConditionsIndex")
jan_2017 = indexer.fit(jan_2017).transform(jan_2017)

encoder = OneHotEncoder(inputCol='ConditionsIndex', outputCol="ConditionsVect")
jan_2017 = encoder.transform(jan_2017).drop('ConditionsIndex')

In [68]:
indexer = StringIndexer(inputCol="period", outputCol="periodIndex")
jan_2017 = indexer.fit(jan_2017).transform(jan_2017)
encoder = OneHotEncoder(inputCol='periodIndex', outputCol="periodVect")
jan_2017 = encoder.transform(jan_2017).drop('periodIndex')

In [69]:
encoder = OneHotEncoder(inputCol='PUDay', outputCol="PUDayVect")
jan_2017 = encoder.transform(jan_2017).drop('PUDay')

In [70]:
jan_2017.show(1)

+--------+---------------+---------+------------+-------------+---------------+--------------+---------+------+--------+---------------+---------------+-------+-----------+-----------+--------+------+---------+----+---------+--------+--------+--------+----------+---------+---------+------+------+------------+-----------------+-----------+----------+-------------+-----------+--------------+---------+----------------+-----------+---------------+---------------+------------------------+----------------+-----------------+---------------------+----------------------+-------------+------------+---------+----------+----------+----------+------------+--------------------+------------+----------+------------+--------------+---------------+---------+---------+----------+----------------+-------------+-------------------+----------+--------------+---------+-----------------+-----------+---------+-----------+---------------+--------------+----------------+-----------------+----------+-------------

In [71]:
indexer = StringIndexer(inputCol="windDir", outputCol="windDirIndex")
jan_2017 = indexer.fit(jan_2017).transform(jan_2017)
encoder = OneHotEncoder(inputCol='windDirIndex', outputCol="windDirVect")
jan_2017 = encoder.transform(jan_2017).drop('windDirIndex')

In [72]:
# Drop any string columns except 'PUZone'
str_names = ['PUZone','tpep_pickup_datetime','tpep_dropoff_datetime','store_and_fwd_flag','PULocationID',\
             'LocationID','PUBorough','PUServiceZone','PUneighbor','PUTime','PUTemptime',\
             'windDir','Events','Condition','period','neighborhood','PUDate', 'PUTempdate']
for col in str_names:
    jan_2017=jan_2017.drop(col)

In [73]:
mapping_df = spark.read.format("csv").load('mappings.csv', header = True)

In [74]:
# sampe = jan_2017.sample(False,0.0001,0).cache()
sampe = jan_2017.join(mapping_df, jan_2017.DLocationID == mapping_df.taxi, "left_outer")

In [75]:
sampe.count()

4230796

In [ ]:
sampe = sampe.withColumn("label", sampe["label"].cast(IntegerType())).drop('DLocationID').drop('taxi')

In [88]:
colum = sampe.schema.names

In [89]:
colum.remove('DLocationID')

In [90]:
assembler = VectorAssembler(
    inputCols=colum,
    outputCol="features")

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1035, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o1248.schema

In [91]:
output = assembler.transform(sampe)

In [ ]:
# train = output.select(["DLocationID","features"])

In [ ]:
# train = train.withColumn("label", train["DLocationID"]).drop("DLocationID")

In [ ]:
# train.select('features').limit(1).collect()

In [ ]:
# train = train.select(["label","features"]).cache()

In [ ]:
jan_2017.show(1)

In [ ]:
train.show(1)

In [94]:
output.select("label", "features").write.save("training.parquet", format="parquet")

In [98]:
output.select("*").write.save("training3.json", format="json")

Py4JJavaError: An error occurred while calling o1515.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply$mcV$sp(FileFormatWriter.scala:215)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:173)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:173)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:65)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:173)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:145)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	at org.apac